<a href="https://colab.research.google.com/github/Fakhre-Alam-Hub/Movie-Recommendation-System/blob/main/Data%20Collection/Preprocessing/.ipython/processing_2020s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Adding Genre And Rating In The DataFrame**

In [1]:
!pip install IMDbpy

     |████████████████████████████████| 307kB 2.6MB/s 


In [2]:
import pandas as pd
import numpy as np
import pickle
import imdb


In [3]:
df_2020s = pd.read_csv("/content/drive/My Drive/dataset/Movie Recommendation System/Dataset/2020s.csv")

In [ ]:
df_2020s.head(7)

,Opening,Opening.1,Title,Director,Cast,Studio,Year
0,JAN,1.0,Ghost Stories,Karan JoharDibakar BanerjeeZoya AkhtarAnurag K...,Sobhita DhulipalaMrunal ThakurAvinash TiwaryJa...,"RSVP Movies, Flying Unicorn Entertainment, Net...",2020
1,JAN,3.0,Bhangra Paa Le,Sneha Taurani,Sunny KaushalRukshar DhillonShriya Pilgaonkar,RSVP Movies,2020
2,JAN,3.0,Sab Kushal Mangal,Karan Vishwanath Kashyap,Akshaye KhannaRiva KishanPriyank Sharma,One Up Entertainment,2020
3,JAN,3.0,Shimla Mirchi,Ramesh Sippy,Hema MaliniRajkummar RaoRakul Preet Singh,Viacom18 Motion Pictures,2020
4,JAN,10.0,Chhapaak,Meghna Gulzar,Deepika PadukoneVikrant Massey,"Fox Star Studios, Ka Productions, Mriga Films",2020
5,JAN,10.0,Tanhaji,Om Raut,Ajay DevgnSaif Ali KhanKajolSharad Kelkar,"Ajay Devgn FFilms, T-Series",2020
6,JAN,17.0,Bunker,Jugal Raja,Abhijeet SinghArindita Kalita,Wagging Tail Entertainment,2020


In [4]:
df_2020s['Title'] = df_2020s['Title'].str.lower()

In [7]:
df_2020s[df_2020s['Title'].isnull()]

,Opening,Opening.1,Title,Director,Cast,Studio,Year
70,21,NaN,NaN,NaN,NaN,NaN,2020
82,NOV,NaN,NaN,NaN,NaN,NaN,2020


In [18]:
df_2020s.dropna(axis=0,how='any',inplace=True)

In [21]:
def get_genres(row):
  io = imdb.IMDb()
  search=io.search_movie(row['Title'])

  for i in range(len(search)):
    try:
      if search[i]['year']==row['Year']:
        id=search[i].movieID
        movie=io.get_movie(id)
        return listToString(movie['genres'])
    except KeyError:
      None

In [22]:
df_2020s['Genres'] = df_2020s.apply(lambda x : get_genres(x),axis=1)

In [23]:
df_2020s.head()

,Opening,Opening.1,Title,Director,Cast,Studio,Year,Genres
0,JAN,1.0,ghost stories,Karan JoharDibakar BanerjeeZoya AkhtarAnurag K...,Sobhita DhulipalaMrunal ThakurAvinash TiwaryJa...,"RSVP Movies, Flying Unicorn Entertainment, Net...",2020,Horror Thriller
1,JAN,3.0,bhangra paa le,Sneha Taurani,Sunny KaushalRukshar DhillonShriya Pilgaonkar,RSVP Movies,2020,Comedy Drama Music
2,JAN,3.0,sab kushal mangal,Karan Vishwanath Kashyap,Akshaye KhannaRiva KishanPriyank Sharma,One Up Entertainment,2020,Romance
3,JAN,3.0,shimla mirchi,Ramesh Sippy,Hema MaliniRajkummar RaoRakul Preet Singh,Viacom18 Motion Pictures,2020,Comedy Romance
4,JAN,10.0,chhapaak,Meghna Gulzar,Deepika PadukoneVikrant Massey,"Fox Star Studios, Ka Productions, Mriga Films",2020,Biography Drama


In [24]:
def get_rating(row):
  io = imdb.IMDb()
  search=io.search_movie(row['Title'])

  for i in range(len(search)):
    try:
      if search[i]['year']==row['Year']:
        id=search[i].movieID
        movie=io.get_movie(id)
        return movie['rating']
    except KeyError:
      None

In [25]:
df_2020s['Rating'] = df_2020s.apply(lambda x : get_rating(x),axis=1)

In [26]:
df_2020s.head()

,Opening,Opening.1,Title,Director,Cast,Studio,Year,Genres,Rating
0,JAN,1.0,ghost stories,Karan JoharDibakar BanerjeeZoya AkhtarAnurag K...,Sobhita DhulipalaMrunal ThakurAvinash TiwaryJa...,"RSVP Movies, Flying Unicorn Entertainment, Net...",2020,Horror Thriller,4.3
1,JAN,3.0,bhangra paa le,Sneha Taurani,Sunny KaushalRukshar DhillonShriya Pilgaonkar,RSVP Movies,2020,Comedy Drama Music,6.1
2,JAN,3.0,sab kushal mangal,Karan Vishwanath Kashyap,Akshaye KhannaRiva KishanPriyank Sharma,One Up Entertainment,2020,Romance,6.1
3,JAN,3.0,shimla mirchi,Ramesh Sippy,Hema MaliniRajkummar RaoRakul Preet Singh,Viacom18 Motion Pictures,2020,Comedy Romance,4.6
4,JAN,10.0,chhapaak,Meghna Gulzar,Deepika PadukoneVikrant Massey,"Fox Star Studios, Ka Productions, Mriga Films",2020,Biography Drama,5.1


In [28]:
row=df_2020s.iloc[0]

In [29]:
row

Opening                                                    JAN
Opening.1                                                    1
Title                                            ghost stories
Director     Karan JoharDibakar BanerjeeZoya AkhtarAnurag K...
Cast         Sobhita DhulipalaMrunal ThakurAvinash TiwaryJa...
Studio       RSVP Movies, Flying Unicorn Entertainment, Net...
Year                                                      2020
Genres                                         Horror Thriller
Rating                                                     4.3
Name: 0, dtype: object

In [30]:
comb = row['Director']+' '+row['Cast']+' '+row['Genres']

In [52]:
df_2020s[df_2020s['Genres'].isnull()]

,Opening,Opening.1,Title,Director,Cast,Studio,Year,Genres,Rating


In [51]:
df_2020s.isnull().sum()

Opening       0
Opening.1     0
Title         0
Director      0
Cast          0
Studio        0
Year          0
Genres        0
Rating       17
dtype: int64

In [53]:
df_2020s['Genres'] = df_2020s['Genres'].fillna(value=' ')

In [54]:
import re

def combine_feature(row):
  comb = row['Director']+' '+row['Cast']+' '+row['Genres']
  lst=re.findall('.[^A-Z]*', comb)

  def listToString(list):
      str1 = " "
      return (str1.join(list))

  convertedStr = listToString(lst)
  return convertedStr.replace('  ',' ')


In [55]:
df_2020s['Combined'] = df_2020s.apply(lambda x: combine_feature(x),axis=1)

In [56]:
df_2020s.head()

,Opening,Opening.1,Title,Director,Cast,Studio,Year,Genres,Rating,Combined
0,JAN,1.0,ghost stories,Karan JoharDibakar BanerjeeZoya AkhtarAnurag K...,Sobhita DhulipalaMrunal ThakurAvinash TiwaryJa...,"RSVP Movies, Flying Unicorn Entertainment, Net...",2020,Horror Thriller,4.3,Karan Johar Dibakar Banerjee Zoya Akhtar Anura...
1,JAN,3.0,bhangra paa le,Sneha Taurani,Sunny KaushalRukshar DhillonShriya Pilgaonkar,RSVP Movies,2020,Comedy Drama Music,6.1,Sneha Taurani Sunny Kaushal Rukshar Dhillon Sh...
2,JAN,3.0,sab kushal mangal,Karan Vishwanath Kashyap,Akshaye KhannaRiva KishanPriyank Sharma,One Up Entertainment,2020,Romance,6.1,Karan Vishwanath Kashyap Akshaye Khanna Riva K...
3,JAN,3.0,shimla mirchi,Ramesh Sippy,Hema MaliniRajkummar RaoRakul Preet Singh,Viacom18 Motion Pictures,2020,Comedy Romance,4.6,Ramesh Sippy Hema Malini Rajkummar Rao Rakul P...
4,JAN,10.0,chhapaak,Meghna Gulzar,Deepika PadukoneVikrant Massey,"Fox Star Studios, Ka Productions, Mriga Films",2020,Biography Drama,5.1,Meghna Gulzar Deepika Padukone Vikrant Massey ...


In [58]:
df_2020s['Combined'].loc[0]

'Karan Johar Dibakar Banerjee Zoya Akhtar Anurag Kashyap Sobhita Dhulipala Mrunal Thakur Avinash Tiwary Janhvi Kapoor Surekha Sikri Raghuvir Yadav Gulshan Devaiah Vijay Varma Pavail Gulati Jyoti Subhash Eva Ameet Pardeshi Kitu Gidwani Sumit Tandon Horror Thriller'

In [60]:
df_2020s['Combined'] = df_2020s['Combined'].str.lower()

In [61]:
df_2020s.to_csv('processed_2020s.csv',index=False)